# The Battle of Neighborhoods

## 1. Introduction 

   One of my friend is going to the University of Chicago for his grad school, and now he is stuggling with fiding a new apartment around the school. He has some basic requirements of the new apartment in Chicago. They are:
   1. The price of the apartment has to be affordable (around $3K/mo.).
   2. It has to be less than 20 mins away (around 0.6 miles, or 1 km) from the subway station by walking. 
   3. He prefers the neighborhood of his new apartment is similar to the neighborhood of his home in Queens, NY.
 
Based on his requirements above, I'm planning to help him find a new accomodation in Chicago by using data analysis techniques. 


## 2. Data

   The address of my friend's home is in 'Flushing, NY'. This address will be the input to the FourSquare API, and the output will be all the venues around this address. These venues will be stored into a dataframe. Then, for the neighborhoods in Chicago, it can be found on the Wikipeida website 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago'. The subway location data can be found in a dataframe, and the coordinates are already provided, 'https://data.cityofchicago.org/Transportation/CTA-System-Information-List-of-L-Stops/8pix-ypme/data'. The table of all the neighborhoods locates in Chicago can be scraped and loaded in to a dataframe easily as well. The most difficult data to obtain is the apartment rental information. However, with some special packages, it can also be stored into the dataframe. For all the coordinates data, it can be found by using the Geocode package. With all these data, the data processing and analyzing procedure can be moved on. 

The process of loading data is showing below.
   

### 2.1 Acquire All the Data


#### 2.1.1 Get 'homeVenues' Data with FourSquare API 

In [1]:
import numpy as np
import pandas as pd

import requests
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

from bs4 import BeautifulSoup
import time
!conda install -c conda-forge selenium

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1

In [2]:
CLIENT_ID = 'O5JVZA3QFOKXT4TVE4D2TYQCYVZ2PFIVK15B4AGYMQLR24EJ' # your Foursquare ID
CLIENT_SECRET = 'QPINCQCN1LR4VZOVRYJSE5D1W1FVG2K34DQJIDVJDLCTSXF2' # your Foursquare Secret
VERSION = '20200426'

In [3]:
home='Flushing, NY'
geolocator = Nominatim()
location = geolocator.geocode(home)
print(location)
homeLati=location.latitude
homeLongi=location.longitude
print(homeLati)
print(homeLongi)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


Flushing, Queens, New York, Queens County, New York, 11354, United States of America
40.7654301
-73.8174291


In [4]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    homeLati, 
    homeLongi, 
    radius, 
    LIMIT)
venue_results = requests.get(url).json()
Venues = venue_results['response']['groups'][0]['items']
Venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4c4f52248b5520a117844b04',
   'name': 'NY Puppy Club',
   'location': {'address': '149-05 Northern Blvd',
    'lat': 40.76540729469897,
    'lng': -73.8171024202533,
    'labeledLatLngs': [{'label': 'display',
      'lat': 40.76540729469897,
      'lng': -73.8171024202533}],
    'distance': 27,
    'postalCode': '11354',
    'cc': 'US',
    'city': 'Flushing',
    'state': 'NY',
    'country': 'United States',
    'formattedAddress': ['149-05 Northern Blvd',
     'Flushing, NY 11354',
     'United States']},
   'categories': [{'id': '5032897c91d4c4b30a586d69',
     'name': 'Pet Service',
     'pluralName': 'Pet Services',
     'shortName': 'Pet Service',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []

In [5]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [6]:
homeVenues = json_normalize(Venues) # flatten JSON
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
homeVenues =homeVenues.loc[:, filtered_columns]
# filter the category for each row
homeVenues['venue.categories'] = homeVenues.apply(get_category_type, axis=1)
# clean columns
homeVenues.columns = [col.split(".")[-1] for col in homeVenues.columns]
print('The shape of \'homeVenues\' is: ', homeVenues.shape)
homeVenues.head(10)

The shape of 'homeVenues' is:  (45, 4)


,name,categories,lat,lng
0,NY Puppy Club,Pet Service,40.765407,-73.817102
1,"Voelker Orth Museum, Bird Sanctuary and Victor...",History Museum,40.763886,-73.819388
2,Hahm Ji Bach - 함지박,Korean Restaurant,40.763022,-73.815042
3,Butcher's Cut,Korean Restaurant,40.765190,-73.820273
4,Kum Sung Chik Naengmyun,Korean Restaurant,40.763122,-73.815091
5,Mapo BBQ,Korean Restaurant,40.762309,-73.814880
6,Coffee Factory,Coffee Shop,40.763125,-73.814341
7,Cafe de Cupping,Coffee Shop,40.765261,-73.814368
8,SGD Tofu House,Korean Restaurant,40.762125,-73.815532
9,7-Eleven,Convenience Store,40.764905,-73.819692


#### 2.1.2 Get Chicago Neighborhoods Data from Wikipedia

In [7]:
# Scrape neighborhood data from Wikipedia
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago')
for i in range(0, len(dfs)):
    print(i)
    #print(dfs[i])

0
             Neighborhood         Community area
0             Albany Park            Albany Park
1         Altgeld Gardens              Riverdale
2           Andersonville              Edgewater
3          Archer Heights         Archer Heights
4           Armour Square          Armour Square
5                 Ashburn                Ashburn
6         Ashburn Estates                Ashburn
7          Auburn Gresham         Auburn Gresham
8             Avalon Park            Avalon Park
9                Avondale               Avondale
10       Avondale Gardens            Irving Park
11      Back of the Yards               New City
12        Belmont Central         Belmont Cragin
13        Belmont Gardens                Hermosa
14        Belmont Heights                Dunning
15        Belmont Terrace                Dunning
16                Beverly                Beverly
17           Beverly View                Ashburn
18          Beverly Woods            Morgan Park
19               B

In [8]:
Chicago=dfs[0]
Chicago.columns=['Neighborhood', 'Brorough']
print('The shape of \'Chicago\' is: ', Chicago.shape)
Chicago.head()

The shape of 'Chicago' is:  (246, 2)


,Neighborhood,Brorough
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square
5,Ashburn,Ashburn
6,Ashburn Estates,Ashburn
7,Auburn Gresham,Auburn Gresham
8,Avalon Park,Avalon Park
9,Avondale,Avondale


In [9]:
Lat=[]
Lgt=[]
count=1
for a in Chicago['Neighborhood']:
    address=a+', Chicago, IL'
    geolocator = Nominatim(user_agent="a")
    location = geolocator.geocode(address)
    if location==None:
        latitude=None
        longitude=None
    else:
        latitude = location.latitude
        longitude = location.longitude
    Lat.append(latitude)
    Lgt.append(longitude)
    
Chicago['Latitude']=Lat
Chicago['Longitude']=Lgt

Chicago.head()

,Neighborhood,Brorough,Latitude,Longitude
0,Albany Park,Albany Park,41.971937,-87.716174
1,Altgeld Gardens,Riverdale,41.654864,-87.600446
2,Andersonville,Edgewater,41.977139,-87.669273
3,Archer Heights,Archer Heights,41.811422,-87.726165
4,Armour Square,Armour Square,41.840033,-87.633107
5,Ashburn,Ashburn,41.747533,-87.711163
6,Ashburn Estates,Ashburn,NaN,NaN
7,Auburn Gresham,Auburn Gresham,41.743387,-87.656042
8,Avalon Park,Avalon Park,41.745035,-87.588658
9,Avondale,Avondale,41.938921,-87.711168


#### 2.1.3 Get Chicago Subway Stops Data with Chicago Data Portal API

In [10]:
# Use API to load subway data
api='https://data.cityofchicago.org/resource/8pix-ypme.json'
results = requests.get(api).json()
#results

[{'stop_id': '30162',
  'direction_id': 'W',
  'stop_name': '18th (54th/Cermak-bound)',
  'station_name': '18th',
  'station_descriptive_name': '18th (Pink Line)',
  'map_id': '40830',
  'ada': True,
  'red': False,
  'blue': False,
  'g': False,
  'brn': False,
  'p': False,
  'pexp': False,
  'y': False,
  'pnk': True,
  'o': False,
  'location': {'latitude': '41.857908', 'longitude': '-87.669147'},
  ':@computed_region_awaf_s7ux': '8',
  ':@computed_region_6mkv_f3dw': '14920',
  ':@computed_region_vrxf_vc4k': '33',
  ':@computed_region_bdys_3d7i': '343',
  ':@computed_region_43wa_7qmu': '26'},
 {'stop_id': '30161',
  'direction_id': 'E',
  'stop_name': '18th (Loop-bound)',
  'station_name': '18th',
  'station_descriptive_name': '18th (Pink Line)',
  'map_id': '40830',
  'ada': True,
  'red': False,
  'blue': False,
  'g': False,
  'brn': False,
  'p': False,
  'pexp': False,
  'y': False,
  'pnk': True,
  'o': False,
  'location': {'latitude': '41.857908', 'longitude': '-87.669147'}

In [11]:
subway_full_data = json_normalize(results)
subway_full_data.head()

,:@computed_region_43wa_7qmu,:@computed_region_6mkv_f3dw,:@computed_region_awaf_s7ux,:@computed_region_bdys_3d7i,:@computed_region_vrxf_vc4k,ada,blue,brn,direction_id,g,...,o,p,pexp,pnk,red,station_descriptive_name,station_name,stop_id,stop_name,y
0,26,14920,8,343,33,True,False,False,W,False,...,False,False,False,True,False,18th (Pink Line),18th,30162,18th (54th/Cermak-bound),False
1,26,14920,8,343,33,True,False,False,E,False,...,False,False,False,True,False,18th (Pink Line),18th,30161,18th (Loop-bound),False
2,1,14924,26,719,56,True,False,False,N,False,...,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30022,35th/Archer (Loop-bound),False
3,1,14924,26,719,56,True,False,False,S,False,...,True,False,False,False,False,35th/Archer (Orange Line),35th/Archer,30023,35th/Archer (Midway-bound),False
4,9,21194,12,25,1,True,False,False,S,True,...,False,False,False,False,False,35th-Bronzeville-IIT (Green Line),35th-Bronzeville-IIT,30214,35-Bronzeville-IIT (63rd-bound),False


In [12]:
subway_full_data.columns

Index([':@computed_region_43wa_7qmu', ':@computed_region_6mkv_f3dw',
       ':@computed_region_awaf_s7ux', ':@computed_region_bdys_3d7i',
       ':@computed_region_vrxf_vc4k', 'ada', 'blue', 'brn', 'direction_id',
       'g', 'location.latitude', 'location.longitude', 'map_id', 'o', 'p',
       'pexp', 'pnk', 'red', 'station_descriptive_name', 'station_name',
       'stop_id', 'stop_name', 'y'],
      dtype='object')

In [13]:
subway=subway_full_data[['stop_id','stop_name','location.latitude','location.longitude']]
print('The shape of \'subway\' is: ', subway.shape)
subway.head()

The shape of 'subway' is:  (300, 4)


,stop_id,stop_name,location.latitude,location.longitude
0,30162,18th (54th/Cermak-bound),41.857908,-87.669147
1,30161,18th (Loop-bound),41.857908,-87.669147
2,30022,35th/Archer (Loop-bound),41.829353,-87.680622
3,30023,35th/Archer (Midway-bound),41.829353,-87.680622
4,30214,35-Bronzeville-IIT (63rd-bound),41.831677,-87.625826


#### 2.1.4 Acquire Rental Data

In [28]:
URL='https://www.realtor.com/apartments/Chicago_IL'
page = requests.get(URL)
page

<Response [200]>

In [29]:
soup=BeautifulSoup(page.content, 'html.parser')
#soup

<!DOCTYPE html>

<html lang="en">
<head>
<link href="https://static.rdc-next-basecamp.rdc.moveaws.com/assets/favicon-144287b492113455703667bc3182768a9b2666765a1cd4beba38cfe40294eccc.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="app-id=com.move.realtor" name="google-play-app"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=5, viewport-fit=cover" name="viewport"/>
<meta content="Find a better way to search for Chicago, IL apartments for rent. Search 15,946 Chicago apartments and more rentals on realtor.com&amp;reg;." name="description">
<meta content="authenticity_token" name="csrf-param">
<meta content="he15CRRwooacNQg1AsbhQjbdMVRBGbdtb5TyZBJN2ySEa6NFVY4grigqfwdUS0wFjKU92sXuC+y9q/dQuGhVpA==" name="csrf-token"/>
<link data-turbolinks-track="true" href="https://static.rdc-next-basecamp.rdc.moveaws.com/assets/application-45912305abab85e4bfce86ca07a634a0d7f69f8bf98ab7e0cb53fd5415647029.css" media="all

In [38]:
results=soup.find(id='srp-list')
#results.prettify()

'<div class="srp-list" id="srp-list">\n <script type="text/javascript">\n  //<![CDATA[\n var txtinAdUnitsList = [];\n var txtinAdUnitsMap = [];\n window.zipcode_of_first_listing = \'\';\n //]]>\n </script>\n <div class="component_ab-sort clearfix">\n  <span id="search-result-count">\n   15,946\n          Rentals\n  </span>\n  <div class="srp-sorting-control" id="srp-sorting-wrap">\n   <fieldset class="srp-sorting-input srp-customselect-input">\n    <label class="font-normal" for="srp-sortby">\n     Sort by\n    </label>\n    <select class="no-show" id="srp-sortby">\n     <option value="5">\n      Freshest Listings\n     </option>\n     <option value="1">\n      Recently Added/Updated\n     </option>\n     <option value="2">\n      Highest Price\n     </option>\n     <option value="3">\n      Lowest Price\n     </option>\n    </select>\n    <div class="srp-custom" id="page-load-sort">\n     <span class="sort-dropdown">\n      <span>\n       Freshest Listings\n      </span>\n     </span>

In [51]:
blocks=results.find_all('li', class_='component_property-card js-component_property-card')
#for i in range (0,3):
    #print(i+1,'\n',blocks[i],'\n\n')
    #print(len(blocks))

1 
 <li class="component_property-card js-component_property-card" data-featured="false" data-listingid="643889363" data-propertyid="7604190222" data-rank="1" data-similar-home-id="similar-home-card" data-url="/realestateandhomes-detail/5252-S-Cornell-Ave_Chicago_IL_60615_M76041-90222" id="1" itemscope="" itemtype="http://schema.org/SingleFamilyResidence">
<div class="seo-wrap hide">
<span itemprop="address" itemscope="" itemtype="http://schema.org/PostalAddress">
<span itemprop="streetAddress">5252 S Cornell Ave</span>
<span itemprop="addressLocality">Chicago</span>
<span itemprop="addressRegion">IL</span>
<span itemprop="postalCode">60615</span>
</span>
<span class="listing-geo" itemprop="geo" itemscope="" itemtype="http://schema.org/GeoCoordinates">
<meta content="41.79984" itemprop="latitude"/>
<meta content="-87.58624" itemprop="longitude"/>
</span>
</div>
<div class="pre-card-wrap">
<div class="broker-info">
<div class="ellipsis">
<span>Provided by</span>
<span class="spon-logo">

In [54]:
urls=[]
for i in range (0, len(blocks)):
    urls.append(blocks[i]['data-url'])

baseURL='https://www.realtor.com'
fullURLs=[]

for j in range (0, len(urls)):
    fullURLs.append(baseURL+urls[j])
for k in range (0,3):
    print(fullURLs[k])

https://www.realtor.com/realestateandhomes-detail/5252-S-Cornell-Ave_Chicago_IL_60615_M76041-90222
https://www.realtor.com/realestateandhomes-detail/540-N-State-St_Chicago_IL_60654_M80549-16295
https://www.realtor.com/realestateandhomes-detail/1457-N-Halsted-St_Chicago_IL_60642_M70005-04056


https://www.realtor.com/realestateandhomes-detail/5252-S-Cornell-Ave_Chicago_IL_60615_M76041-90222
https://www.realtor.com/realestateandhomes-detail/540-N-State-St_Chicago_IL_60654_M80549-16295
https://www.realtor.com/realestateandhomes-detail/1457-N-Halsted-St_Chicago_IL_60642_M70005-04056
